# Search Engine

This lab is about starting from scratch, and hence having to structure the code yourself. A specification for the program you are to implement is given below - do pay attention as misunderstandings may cost you marks! Being precise is a necessary skill for a programmer. To give an executive summary, you are to code a search engine for recipes. A data set has been provided. The search engine is to be pretty basic, returning all recipes that contain all of the provided keywords. However, the user can choose from a number of orderings depending on their food preferences, which you need to support.

## Marking and submission

These lab exercises are marked, and contribute to your final grade. This lab exercise has 20 marks to earn, equivalent to 12% of your final grade.

Please submit your completed workbook to the auto marker before 2021-11-14 20:00 GMT. The workbook you submit must be an .ipynb file, which is saved into the directory you're running Jupyter; alternatively you can download it from the menu above using `File -> Download As -> Notebook (.ipynb)`. Remember to save your work regularly (`Save and checkpoint` in the `File` menu, the icon of a floppy disk, or `Ctrl-S`). It is wise to verify it runs to completion with _Restart & Run All_ before submission.

You must comply with the universities plagiarism guidelines: http://www.bath.ac.uk/library/help/infoguides/plagiarism.html

## Specification

The system must provide a function ``search``, with the following specification:
```
def search(query, ordering = 'normal', count = 10):
  ...
```

It `print`s out the results of the search, subject to the following rules:
1. It selects from the set of all recipes that contain __all__ of the words in the query (the positions/order of the words in the recipe are to be ignored).
2. It orders them based on the provided ordering (a string, meaning defined below).
3. It `print`s the top `count` matches only, preserving the order from best to worst. Must `print` just their title, one per line. Must be less than `count` if the search is specific enough that less than `count` recipes match.

As an aside, do not worry about memory usage. If duplicating some data can make your code faster/neater then feel free.



### Data set

A file, `recipes.json` is provided, containing 17K recipes. It can be parsed into a Python data structure using the [`json`](https://docs.python.org/3/library/json.html) module. It is a list, where each recipe is a dictionary containing various keys:
* `title` : Name of recipe; you can assume these are unique
* `categories` : A list of tags assigned to the recipe
* `ingredients` : What is in it, as a list
* `directions` : List of steps to make the recipe
* `rating` : A rating, out of 5, of how good it is
* `calories` : How many calories it has
* `protein` : How much protein is in it
* `fat` : How much fat is in it

Note that the data set was obtained via web scrapping and hence is noisy - every key except for `title` is missing from at least one recipe. Your code will need to cope with this.

You will probably want to explore the data before starting, so you have an idea of what your code has to deal with.

Data set came from https://www.kaggle.com/hugodarwood/epirecipes/version/2 though note it has been cleaned it up, by deleting duplicates and removing the really dodgy entries.



### Search

The search should check the following parts of the recipe (see data set description below):
* `title`
* `categories`
* `ingredients`
* `directions`

For instance, given the query "banana cheese" you would expect "Banana Layer Cake with Cream Cheese Frosting" in the results. Note that case is to be ignored ("banana" matches "Banana") and the words __do not__ have to be next to one another, in the same order as the search query or even in the same part of the recipe ("cheese" could appear in the title and "banana" in the ingredients). However, all words in the search query __must__ appear somewhere.



### Tokenisation

This is the term for breaking a sentence into each individual word (token). Traditionally done using regular expressions, and Python does have the `re` module, but there is no need to do that here (`re` can be quite fiddly). For matching words your tokenisation must follow the following steps:
1. Convert all punctuation and digits into spaces. For punctuation use the set in [`string.punctuation`](https://docs.python.org/3/library/string.html#string.punctuation), for digits [`string.digits`](https://docs.python.org/3/library/string.html#string.digits). You may find [`translate()`](https://docs.python.org/3/library/stdtypes.html#str.translate) interesting!
2. [`split()`](https://docs.python.org/3/library/stdtypes.html#str.split) to extract individual tokens.
3. Ignore any token that is less than $3$ characters long.
4. Make tokens lowercase.

When matching words for search (above) or ordering (below) it's only a match if you match an entire token. There are many scenarios where this simple approach will fail, but it's good enough for this exercise. The auto marker will be checking the above is followed! When doing a search the code should ignore terms in the search string that fail the above requirements.



### Ordering

There are three ordering modes to select from, each indicated by passing a string to the `search` function:
* `normal` - Based simply on the number of times the search terms appear in the recipe. A score is calculated and the order is highest to lowest. The score sums the following terms (repeated words are counted multiple times, i.e. "cheese cheese cheese" is $3$ matches to "cheese"):
    * $8 \times$ Number of times a query word appears in the title
    * $4 \times$ Number of times a query word appears in the categories
    * $2 \times$ Number of times a query word appears in the ingredients
    * $1 \times$ Number of times a query word appears in the directions
    * The `rating` of the recipe (if not available assume $0$)

* `simple` - Tries to minimise the complexity of the recipe, for someone who is in a rush. Orders to minimise the number of ingredients multiplied by the numbers of steps in the directions.

* `healthy` - Order from lowest to highest by this cost function:
$$\frac{|\texttt{calories} - 510n|}{510} + 2\frac{|\texttt{protein} - 18n|}{18} + 4\frac{|\texttt{fat} - 150n|}{150}$$
Where $n \in \mathbb{N}^+$ is selected to minimise the cost ($n$ is a positive integer and $n=0$ is not allowed). This can be understood in terms of the numbers $510$, $18$ and $150$ being a third of the recommended daily intake (three meals per day) for an average person, and $n$ being the number of whole meals the person gets out of cooking/making the recipe. So this tries to select recipes that neatly divide into a set of meals that are the right amount to consume for a healthy, balanced diet. Try not to overthink the optimisation of $n$, as it's really quite simple to do!

To clarify the use of the ordering string, to get something healthy that contains cheese you might call `search('cheese', 'healthy')`. In the case of a recipe that is missing a key in its dictionary the rules are different for each search mode:
* `normal` - Consider a missing entry in the recipe (e.g. no `ingredients` are provided) to simply mean that entry can't match any search words (because it has none!), but the item is still eligible for inclusion in the results, assuming it can match the search with a different entry.
* `simple` - If a recipe is missing either `ingredients` or `directions` it is dropped from such a search result. Because the data is messy if either of these lists is of length $1$ it should be assumed that the list extraction has failed and the recipe is to also be dropped from the search results.
* `healthy` - If any of `calories`, `protein` or `fat` is missing the recipe should be dropped from the result.



### Extra

You may find the [_inverted index_](https://en.wikipedia.org/wiki/Inverted_index) interesting. It's a data structure used by search engines. For each word a user may search for this contains a list of all documents (recipes) that contain the word. This may take a little effort to understand, but the resulting code will be both faster and neater.

## Advice

* Don't just start coding: Make a plan and work out what you intend to do.
* Think about structure, as messy code leads to mistakes.
* Plan your data structures. Don't be afraid to use sheets of paper if that works for you!
* Don't duplicate code, put it in a function/method instead.

* Divide and conquer. Break the system into parts that can implemented with minimal dependency on the rest. Functions or OOP are both suitable for doing this.
* Test early. Verify the individual parts work before trying to combine them. Factor this into the order you implement the parts of the system - don't implement something you are going to struggle to test before implementing, and verifying, other parts.
* Do not try and do a 'big bang', where you get it all working at once. Instead, get it working with features missing, then add those features in, one at a time.

* Keep things as simple as possible. Avoid long functions/methods.
* Include comments, as a form of planning and for your own sanity!
* Regularly reset the kernel and rerun the entire workbook. It is very easy to break something but not notice, because the correct version remains in memory.

## Marks
* __20 marks__: For many different _unit tests_ that will be run on `search`. They will cover all of the details in the above specification!
    * One test checks that it's faster than $0.1$ seconds on average (on the computer which runs the auto marker, which is quite fast) to do a search, so try not to be too inefficient (it ignores any time your notebook spends building data structures to be used by `search`). Note that the validation implementation comes in at $0.001$ seconds per search (after $5.5$ seconds of preparation), so this is very achievable!
    * You may want to look into Python's `set()` object, as it is useful for _one_ of the possible ways to implement this.
    * There will be sorting. The [Sorting how to](https://docs.python.org/3/howto/sorting.html) may help.
    * The auto marker does give some feedback, and you can run it as many times as you want. Don't be afraid to test an incomplete or semi-broken version of your code if stuck or unsure. It may help!
    * The validation implementation is 104 lines of code split over 5 cells (including white space for clarity and comments). Coded by someone who probably has much more experience than you, so you shouldn't aim to match this, but it's a good clue: If you find yourself at 500 lines you may want to stop and think some more! (line count does not include testing code, which is about the same amount again)

In [1]:
import json
import string
import numpy as np
from collections import OrderedDict
from operator import itemgetter
import pandas as pd


In [2]:
f = open('recipes.json',)
data = json.load(f)

In [3]:
def token(query):
    
    for i in string.punctuation:
        query=query.replace(i,' ')
    for i in string.digits:
        query=query.replace(i,' ')

    query=query.split(' ')
    x=[]
    #print(query)
    for i in query:
        #print(len(i))
        if len(i)>=3:
            x.append(i.lower())
    #print(x)
    return x

In [4]:
def token3(query):

    x=[]
    #print(query)
    for j in query:
        for i in string.punctuation:
            j=j.replace(i,' ')
        for i in string.digits:
            j=j.replace(i,' ')
        
        j=j.split(' ')
        #print(j)
        for i in j:
            #print(len(i))
            if len(i)>=3:
                x.append(i.lower())
            #print(x)
    return x

In [5]:
def simple(query):
    
    if( len(query['ingredients'])>1 and len(query['directions'])>1):
        l=len(query['ingredients'])
        l=l*len(query['directions'])
    else:
        l=-1
    return l



In [6]:
def healthy(row):
    m=1
    if( row['calories']!=-1.0 and row['protein']!=-1.0 and row['fat']!=-1.0):
        l=(np.fabs(row['calories']-(510*m)))/510
        l=l+2*(np.fabs(row['protein']-(18*m)))/18
        l=l+4*(np.fabs(row['fat']-(150*m)))/150
    else:
        l=-1
    return l

In [7]:
df = pd.DataFrame(columns=['title','categories','ingredients','directions','rating','calories','protein','fat','title_token',
                          'categ_token','ingred_token','direc_token','simple'])



df = df.append(data)

df[['categories','ingredients','directions']]=df[['categories','ingredients','directions']].fillna('[]')
df[['calories','protein','fat']]=df[['calories','protein','fat']].fillna(-1.0)

df['title_token'] = df['title'].apply(token)          

df['categ_token'] = df['categories'].apply(token3)
df['ingred_token'] = df['ingredients'].apply(token3)
df['direc_token'] = df['directions'].apply(token3)


In [8]:
df['simple']= df.apply (lambda row: simple(row), axis=1)

In [9]:
df['healthy']= df.apply (lambda row: healthy(row), axis=1)

In [10]:
def title_count(row):
    t_count=set()
    t_c={}
    for i in row['title_token']:
        if(i not in t_count):
            t_count.add(i)
            t_c[i]=1
        else:
            t_c[i]+=1
    return t_c
df['title_count']= df.apply (lambda row: title_count(row), axis=1)

In [11]:
def categ_count(row):
    t_count=set()
    t_c={}
    for i in row['categ_token']:
        if(i not in t_count):
            t_count.add(i)
            t_c[i]=1
        else:
            t_c[i]+=1
    return t_c
df['categ_count']= df.apply (lambda row: categ_count(row), axis=1)

In [12]:
def ingred_count(row):
    t_count=set()
    t_c={}
    for i in row['ingred_token']:
        if(i not in t_count):
            t_count.add(i)
            t_c[i]=1
        else:
            t_c[i]+=1
    return t_c
df['ingred_count']= df.apply (lambda row: ingred_count(row), axis=1)

In [13]:
def direc_count(row):
    t_count=set()
    t_c={}
    for i in row['direc_token']:
        if(i not in t_count):
            t_count.add(i)
            t_c[i]=1
        else:
            t_c[i]+=1
    return t_c
df['direc_count']= df.apply (lambda row: direc_count(row), axis=1)

In [14]:
t_table= {}
t_set=set()
c_table= {}
c_set=set()
i_table= {}
i_set=set()
d_table= {}
d_set=set()
for index,row in df.iterrows():
    for i in row['title_token']:
        if i not in t_set:
            t_set.add(i)
            y=[index]
            y=set(y)
            t_table[i]=y
        else:
            y=t_table[i]
            y.add(index)
            t_table[i]=y
            
    for i in row['categ_token']:
        if i not in c_set:
            c_set.add(i)
            y=[index]
            y=set(y)
            c_table[i]=y
        else:
            y=c_table[i]
            y.add(index)
            c_table[i]=y
    
    for i in row['ingred_token']:
        if i not in i_set:
            i_set.add(i)
            y=[index]
            y=set(y)
            i_table[i]=y
        else:
            y=i_table[i]
            y.add(index)
            i_table[i]=y
            
    for i in row['direc_token']:
        if i not in d_set:
            d_set.add(i)
            y=[index]
            y=set(y)
            d_table[i]=y
        else:
            y=d_table[i]
            y.add(index)
            d_table[i]=y
            

In [15]:
def find(row,j):
    return row.get(j,0)

In [16]:
df

,title,categories,ingredients,directions,rating,calories,protein,fat,title_token,categ_token,ingred_token,direc_token,simple,healthy,title_count,categ_count,ingred_count,direc_count
0,"""Adult"" Pimiento Cheese","[Cheese, Vegetable, No-Cook, Vegetarian, Quick...","[2 or 3 large garlic cloves, a 2-ounce jar dic...",[Force garlic through a garlic press into a la...,3.125,-1.0,-1.0,-1.0,"[adult, pimiento, cheese]","[cheese, vegetable, cook, vegetarian, quick, e...","[large, garlic, cloves, ounce, jar, diced, pim...","[force, garlic, through, garlic, press, into, ...",14,-1.000000,"{'adult': 1, 'pimiento': 1, 'cheese': 1}","{'cheese': 1, 'vegetable': 1, 'cook': 1, 'vege...","{'large': 1, 'garlic': 1, 'cloves': 1, 'ounce'...","{'force': 1, 'garlic': 2, 'through': 1, 'press..."
1,"""Blanketed"" Eggplant","[Tomato, Vegetable, Appetizer, Side, Vegetaria...","[8 small Japanese eggplants, peeled, 16 large ...",[Place eggplants on double thickness of paper ...,3.750,1386.0,9.0,133.0,"[blanketed, eggplant]","[tomato, vegetable, appetizer, side, vegetaria...","[small, japanese, eggplants, peeled, large, fr...","[place, eggplants, double, thickness, paper, t...",50,3.170980,"{'blanketed': 1, 'eggplant': 1}","{'tomato': 1, 'vegetable': 1, 'appetizer': 1, ...","{'small': 1, 'japanese': 1, 'eggplants': 1, 'p...","{'place': 1, 'eggplants': 6, 'double': 1, 'thi..."
2,"""Bloody Mary"" Tomato Toast with Celery and Hor...","[Condiment/Spread, Tomato, Appetizer, Kid-Frie...","[1 lemon, zested, juiced, 1/2 shallot, finely ...","[Combine lemon zest, lemon juice, shallot, tom...",5.000,189.0,2.0,16.0,"[bloody, mary, tomato, toast, with, celery, an...","[condiment, spread, tomato, appetizer, kid, fr...","[lemon, zested, juiced, shallot, finely, chopp...","[combine, lemon, zest, lemon, juice, shallot, ...",28,5.980523,"{'bloody': 1, 'mary': 1, 'tomato': 1, 'toast':...","{'condiment': 1, 'spread': 1, 'tomato': 1, 'ap...","{'lemon': 1, 'zested': 1, 'juiced': 1, 'shallo...","{'combine': 2, 'lemon': 2, 'zest': 1, 'juice':..."
3,"""Brown on Blonde"" Blondies","[Cookbook Critic, Dessert, Brownie, Nut, Walnu...","[170 grams unsalted butter, 100 grams walnuts,...",[Crank the oven to 350ºF. Coat the insides of ...,0.000,321.0,5.0,18.0,"[brown, blonde, blondies]","[cookbook, critic, dessert, brownie, nut, waln...","[grams, unsalted, butter, grams, walnuts, gram...","[crank, the, oven, coat, the, insides, inch, s...",66,5.335033,"{'brown': 1, 'blonde': 1, 'blondies': 1}","{'cookbook': 1, 'critic': 1, 'dessert': 1, 'br...","{'grams': 8, 'unsalted': 1, 'butter': 2, 'waln...","{'crank': 1, 'the': 20, 'oven': 1, 'coat': 1, ..."
4,"""California Roll"" Salad","[Salad, Ginger, Rice, Vegetable, Side, Vegetar...","[1 1/2 cups long-grain rice, 1/4 cup plus 3 ta...",[Into a large saucepan of salted boiling water...,4.375,369.0,9.0,13.0,"[california, roll, salad]","[salad, ginger, rice, vegetable, side, vegetar...","[cups, long, grain, rice, cup, plus, tablespoo...","[into, large, saucepan, salted, boiling, water...",133,4.929804,"{'california': 1, 'roll': 1, 'salad': 1}","{'salad': 1, 'ginger': 1, 'rice': 1, 'vegetabl...","{'cups': 1, 'long': 1, 'grain': 1, 'rice': 2, ...","{'into': 4, 'large': 2, 'saucepan': 2, 'salted..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17729,Zucchini-Pecan Cake with Cream Cheese Frosting,"[Cake, Mixer, Egg, Dessert, Bake, Kentucky Der...","[Nonstick vegetable oil spray, 1 1/2 cups all ...",[Position rack in center of oven and preheat t...,4.375,453.0,6.0,31.0,"[zucchini, pecan, cake, with, cream, cheese, f...","[cake, mixer, egg, dessert, bake, kentucky, de...","[nonstick, vegetable, oil, spray, cups, all, p...","[position, rack, center, oven, and, preheat, l...",90,4.618431,"{'zucchini': 1, 'pecan': 1, 'cake': 1, 'with':...","{'cake': 1, 'mixer': 1, 'egg': 1, 'dessert': 1...","{'nonstick': 1, 'vegetable': 1, 'oil': 2, 'spr...","{'position': 1, 'rack': 2, 'center': 2, 'oven'..."
17730,"Zucchini-Wrapped Halibut ""

In [17]:
def order(df,query,ordering):
    score={}
    b=set()
    if(ordering=='normal'):  
            #print('index',i)
            
            flag=0
            for j in query:
                if(flag==0):
                    a=set.union(t_table.get(j,b),c_table.get(j,b),i_table.get(j,b),d_table.get(j,b))
                    flag=1
                else:
                    a=set.intersection(a,set.union(t_table.get(j,b),c_table.get(j,b),i_table.get(j,b),d_table.get(j,b)))
                    
                #print(len(a))
            #if(len(a)>0):
            #print(a)
            sub=df.iloc[list(a)]
            #print(sub)
            x=0
            for j in query:
                #print(j)
                x+= 8*sub.apply (lambda row: find(row['title_count'],j), axis=1)
                x+=4* sub.apply (lambda row: find(row['categ_count'],j), axis=1)
                x+=2*sub.apply (lambda row: find(row['ingred_count'],j), axis=1)
                x+= sub.apply (lambda row: find(row['direc_count'],j), axis=1)
            x=x+sub['rating']
            #print(sub['result'])
            
            score=dict(zip(sub['title'], x))
       
            
    if(ordering=='simple'):
        flag=0
        for j in query:
            if(flag==0):
                a=set.union(t_table.get(j,b),c_table.get(j,b),i_table.get(j,b),d_table.get(j,b))
                flag=1
            else:
                a=set.intersection(a,set.union(t_table.get(j,b),c_table.get(j,b),i_table.get(j,b),d_table.get(j,b)))
                
        sub=df.iloc[list(a)]
        sub=sub[sub['simple']!=-1]
        score=dict(zip(sub['title'], sub['simple']))
                
    if(ordering=='healthy'):
        flag=0
        for j in query:
            if(flag==0):
                a=set.union(t_table.get(j,b),c_table.get(j,b),i_table.get(j,b),d_table.get(j,b))
                flag=1
            else:
                a=set.intersection(a,set.union(t_table.get(j,b),c_table.get(j,b),i_table.get(j,b),d_table.get(j,b)))
            
        sub=df.iloc[list(a)]
        sub=sub[sub['healthy']!=-1.0]
        score=dict(zip(sub['title'], sub['healthy']))
    return score

In [18]:
def search(query, ordering = 'normal', count = 10):

    query=set(token(query))
   

    score=order(df,query,ordering)
    
    
    if(ordering=='normal'):
        score = OrderedDict(reversed(sorted(score.items(), key=itemgetter(1))))
    elif(ordering=='simple' or ordering=='healthy'):
        score = OrderedDict(sorted(score.items(), key=itemgetter(1)))
    
    #print(score) 
    
    if(len(score)<count):
        count=len(score)
            
    for key,value in score.items():
        print(key)
        #print(value)
        count=count-1
        if(count==0):
            break

In [37]:
import time
print(time.time())
search('bread','normal',5)
print(time.time())

1636911411.9648
Sticky Date and Almond Bread Pudding with Amaretto Zabaglione 
Dry Corn Bread for Bread Pudding 
Ham and Fresh Peach Chutney on Corn Bread 
Prune, Apple, and Chestnut Bread Pudding 
Apple and Maple Bread Pudding 
1636911412.0974448


In [36]:
 search('peanut butter', 'normal', 7)

Chocolate Cake with Milk Chocolate-Peanut Butter Frosting and Peanut Butter Brittle 
Peanut Butter Tart with Caramel-Peanut Glaze 
Peanut Butter Cake with Chocolate-Peanut Butter Icing 
Peanut Butter Banana Cream Pie 
Banana Cupcakes with Peanut Butter Frosting 
Chocolate-Peanut Butter Cake with Cream Cheese and Butterfinger Frosting 
Peanut Butter Cheesecake with Peanut Brittle 


In [21]:
# search('lemon honey cake', 'normal', 4)

In [22]:
# import time
# print(time.time())
# search('baked potato', 'simple', 3)
# print(time.time())

In [23]:
#search('pie', 'healthy', 20)

In [24]:
#search('tempura fish', 'healthy', 10)

In [25]:
#search('mousse', 'simple', 1)

In [26]:
#search('blue cheese salad', 'healthy', 10)

In [27]:
#x

In [28]:
f = open('recipes.json',)
data = json.load(f)

for i in data:
    if i['title']=='Sweet Avocado Mousse ':
        print(i)

{'title': 'Sweet Avocado Mousse ', 'categories': ['Milk/Cream', 'Dairy', 'Dessert', 'Quick & Easy', 'Avocado', 'Chill', 'Gourmet'], 'ingredients': ['2 chilled firm-ripe California avocados', '1 1/2 tablespoons fresh lemon juice', '3/4 cup whole milk', '1/2 cup sugar', '1/2 cup well-chilled heavy cream'], 'directions': ['Halve and pit avocados and scoop flesh into a food processor. Add lemon juice, milk, sugar, and a pinch salt and puree until completely smooth. In a bowl beat cream until it just holds stiff peaks. Fold avocado mixture into whipped cream gently but thoroughly.'], 'rating': 3.75, 'calories': 338.0, 'protein': 4.0, 'fat': 22.0}


In [29]:
# df[df['simple']==-1]